### Author : Antonio Sirica - Course : Computational Intelligence

In [ ]:
import logging
from itertools import combinations
import pandas as pd
import numpy as np
%pip install geopy
from geopy.distance import geodesic #geodesic distance
%pip install networkx
import networkx as nx #library for graphs
from random import random, seed, shuffle
from dataclasses import dataclass
import random
from icecream import ic

logging.basicConfig(level=logging.DEBUG)

# Instance 1

In [ ]:
CITIES = pd.read_csv('cities/italy.csv', header=None, names=['name', 'lat', 'lon'])


# Instance 2

In [108]:
CITIES = pd.read_csv('cities/us.csv', header=None, names=['name', 'lat', 'lon'])


# Instance 3

In [ ]:
CITIES = pd.read_csv('cities/china.csv', header=None, names=['name', 'lat', 'lon'])


# Instance 4

In [ ]:
CITIES = pd.read_csv('cities/vanuatu.csv', header=None, names=['name', 'lat', 'lon'])


# Instance 5

In [ ]:
CITIES = pd.read_csv('cities/russia.csv', header=None, names=['name', 'lat', 'lon'])


# Execute from here with the chosen instance

In [ ]:
DIST_MATRIX = np.zeros((len(CITIES), len(CITIES)))
for c1, c2 in combinations(CITIES.itertuples(), 2): # unique pairs of cities
    DIST_MATRIX[c1.Index, c2.Index] = DIST_MATRIX[c2.Index, c1.Index] = geodesic(
        (c1.lat, c1.lon), (c2.lat, c2.lon)
    ).km #calculates the geodesic (surface distance on the Earth's sphere) in kilometers between the coordinates of c1 and c2
    #distances are stored symmetrically in DIST_MATRIX 
CITIES.head()

#df=pd.DataFrame(DIST_MATRIX)
#df.head()


In [110]:
def tsp_cost(tsp):
    assert tsp[0] == tsp[-1]
    #assert set(tsp) == set(range(len(CITIES)))

    tot_cost = 0
    for c1, c2 in zip(tsp, tsp[1:]): #two separate sequences that are being zipped together to create pairs of elements.
        tot_cost += DIST_MATRIX[c1, c2]
    return tot_cost

# Random Initialization of an optimal TSP


In [100]:
POPULATION_SIZE = 100
OFFSPRING_SIZE = 10_000

city = 0
vec = list(range(1, len(CITIES)))  # Convert range to a list so it can be shuffled
random.shuffle(vec)                # Shuffle vec in place
tsp = list()
tsp = [city] + vec + [city]        # Create tsp by starting and ending with city


# Greedy Initialization of an optimal TSP 

In [ ]:
POPULATION_SIZE = 10
OFFSPRING_SIZE = 5_000

visited = np.full(len(CITIES), False)
dist = DIST_MATRIX.copy()
city = 0
visited[city] = True
tsp = list()
tsp.append(int(city))
while not np.all(visited):
    dist[:, city] = np.inf  #marks the city as "visited" by making it impossible to be selected again
    closest = np.argmin(dist[city])
    logging.debug(
        f"step: {CITIES.at[city,'name']} -> {CITIES.at[closest,'name']} ({DIST_MATRIX[city,closest]:.2f}km)"
    )
    visited[closest] = True
    city = closest #current city
    tsp.append(int(city))
logging.debug(
    f"step: {CITIES.at[tsp[-1],'name']} -> {CITIES.at[tsp[0],'name']} ({DIST_MATRIX[tsp[-1],tsp[0]]:.2f}km)"
)
tsp.append(tsp[0]) #back to starting



#randomized_tsp = tsp[45:len(tsp)-1]
#random.shuffle(randomized_tsp) #shuffle works in place
#tsp[45:len(tsp)-1] = randomized_tsp


logging.info(f"result: Found a path of {len(tsp)-1} steps, total length {tsp_cost(tsp):.2f}km")

# Evolutionary Algorithm

In [ ]:

#dataclass container of different field, printable easily
@dataclass 
class Individual:
    genome: list
    fitness: float = None


def fitness(individual):
    return -float(tsp_cost(individual.genome))

def parent_selection(population):
    candidates = sorted(np.random.choice(population, 2), key=lambda e: e.fitness, reverse=True)
    return candidates[0]


def xover(p1: Individual, p2: Individual):
    
    m = random.randint(1, len(p1.genome) - 1)
    #ic(m)
    genome = p1.genome.copy()
    genome[m] = p2.genome[m]
    return Individual(genome=genome)


def inver_over(p1: Individual, p2: Individual) -> Individual:
    genome_no_first1 = p1.genome[1:-1]
    genome_no_first2 = p2.genome[1:-1]

    # Select a random gene from the first parent's middle section
    m1 = random.randint(0, len(genome_no_first1) - 1)
    gene_from_first = genome_no_first1[m1]

    # Find the position of the same gene in the second parent, or next valid edge
    try:
        m2 = genome_no_first2.index(gene_from_first) + 1
        if m2 >= len(genome_no_first2):
            m2 = 0  # Wrap-around if it exceeds bounds
    except ValueError:
        return p1  # Fallback if gene not found in second parent

    edge_from_second = genome_no_first2[m2]

    # Find the position of this edge in the first parent's genome
    try:
        m3 = genome_no_first1.index(edge_from_second)
    except ValueError:
        return p1  # Fallback if edge not found

    # Sort the indices to reverse elements between them
    idx1, idx2 = sorted([m1, m3])
    #ic(idx1,idx2)

    # Selected and between-elements for new recombination
    selected_elements = [genome_no_first1[idx1], genome_no_first1[idx2]]
    elements_between = genome_no_first1[idx1 + 1: idx2]
    elements_between.reverse()

    # Untouched elements around the selected indices
    untouched_elements = genome_no_first1[:idx1] + genome_no_first1[idx2 + 1:]

    # Construct the new genome, preserving the first and last element of parent 1
    new_array = [p1.genome[0]] + list(dict.fromkeys(untouched_elements + selected_elements + elements_between)) + [p1.genome[-1]]

    return Individual(genome=new_array)

# Example usage
first_genome = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 1]
second_genome = [1, 4, 6, 8, 2, 5, 3, 7, 9, 10, 11, 1]

result = inver_over(Individual(genome=first_genome), Individual(genome=second_genome))

print("Resulting List:", result.genome)
print("Length of Resulting Genome:", len(result.genome))







In [ ]:
def inversion_mutation(genome):
    """Applies inversion mutation on a genome sequence.
       Two loci are selected, and the alleles between them are inverted.
       The first and last elements remain untouched.
    """
    # Choose two random indices, excluding the first and last elements
    idx1, idx2 = sorted(random.sample(range(1, len(genome) - 1), 2))
    
    # Invert the elements between idx1 and idx2
    inverted_segment = genome[idx1:idx2 + 1][::-1]
    
    # Create the new genome by preserving the first, last, and non-inverted segments
    new_genome = genome[:idx1] + inverted_segment + genome[idx2 + 1:]
    
    return new_genome

# Example usage
original_genome = [1, 2, 3, 4, 5, 6, 7]
result = inversion_mutation(original_genome)
print("Original List:", original_genome)
print("Resulting List:", result)

## Starting Population

In [114]:


first_individual = Individual(genome=tsp)
first_individual.fitness = fitness(first_individual)

num_individuals = 1  # Start with the first individual
f = tsp_cost(first_individual.genome)  # Initial cost for the first individual

# Create a list to store individuals in the population
population = list()

# Mutate until the population reaches the specified size
while num_individuals < POPULATION_SIZE:
    # Create a copy of the first individual for mutation
    copy = first_individual.genome.copy()

    # Perform the mutation to generate a new individual
    new_individual_genome = inversion_mutation(copy)

    # Create and evaluate the new individual
    new_individual = Individual(genome=new_individual_genome)
    new_individual.fitness = fitness(new_individual)

    # Append only if the new individual's fitness exceeds the threshold
    if new_individual.fitness > first_individual.fitness:
        population.append(new_individual)
        num_individuals += 1
        #ic(num_individuals, new_individual.fitness)
        


## Offspring step

In [ ]:


offspring = list()
for _ in range(OFFSPRING_SIZE):
    i1 = parent_selection(population)
    i2 = parent_selection(population)
    #o = xover(i1, i2)
    o = inver_over(i1, i2)
    offspring.append(o)

for i in offspring:
    i.fitness = fitness(i)

population.extend(offspring)
population.sort(key=lambda i: i.fitness, reverse=True)
population = population[:POPULATION_SIZE]


print(population[0])
ic(len(population[0].genome))
